In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

#### Create class

In [2]:
class Worksheet:
    
    def __init__(self, data, W, K, Vol, r, T):
        self.data = pd.read_csv(data)
        self.W = W
        self.K = K
        self.Vol = Vol
        self.r = r
        self.T = T

        
        #Deal with Black Schole
        self.data["Expiration"] = pd.Series(range(self.T,  -1, -1)) / 52 #52 weeks == 1 year, create new column for self.data
        
        Call = np.zeros(len(self.data))
        for i in range(len(self.data)):
            if self.data.Expiration[i] > 0:
                d1 = (np.log(self.data.Weekly_Price[i] / self.K) + (self.r + 0.5 * (self.Vol**2)) * self.data.Expiration[i]) / (self.Vol * np.sqrt(self.data.Expiration[i]))
                d2 = d1 - self.Vol * np.sqrt(self.data.Expiration[i])       
            else:  
                if self.data.Weekly_Price[i] >= self.K:
                    d1 = 99999
                    d2 = d1 - self.Vol * np.sqrt(self.data.Expiration[i])
                else:
                    d1 = -99999
                    d2 = d1 - self.Vol * np.sqrt(self.data.Expiration[i])
            Call[i] = self.data.Weekly_Price[i] * norm.cdf(d1) - np.exp(-self.r * self.data.Expiration[i]) * self.K * norm.cdf(d2)
        
        self.data["Call_Black_Schole"] = Call
        
        
        #Deal with Delta 
        Delta = np.zeros(len(self.data))
        for i in range(len(self.data)):
            if self.data.Expiration[i] > 0:
                d1 = (np.log(self.data.Weekly_Price[i] / self.K) + (self.r + 0.5 * (self.Vol**2)) * self.data.Expiration[i]) / (self.Vol * np.sqrt(self.data.Expiration[i]))   
            else:   
                if self.data.Weekly_Price[i] >= self.K:
                    d1 = 99999   
                else:
                    d1 = -99999
            Delta[i] = norm.cdf(d1)
    
        self.data["Delta"] = Delta

        
        #Deal with Cost
        self.data["Share_Netural"] = self.W * self.data["Delta"]
        self.data["Share_Purchased"] = self.data["Share_Netural"].diff(1)
        self.data["Share_Purchased"].values[0] = self.data["Share_Netural"][0]
        self.data["Cost_of_Share"] = self.data["Share_Purchased"] * self.data["Weekly_Price"]
        
        self.data["Interest_expense"] = np.zeros(len(self.data))
        self.data["Cumulative_Cost"] = np.zeros(len(self.data))
        for i in range(len(self.data)):
            if i == 0:
                self.data.loc[i, "Cumulative_Cost"] = self.data.loc[i, "Cost_of_Share"]
                self.data.loc[i, "Interest_expense"] = self.data.loc[i, "Cumulative_Cost"] * (self.r * 1/52)
            else:
                self.data.loc[i, "Cumulative_Cost"] = self.data.loc[i-1, "Cumulative_Cost"] + self.data.loc[i, "Cost_of_Share"] +self.data.loc[i-1, "Interest_expense"]
                self.data.loc[i, "Interest_expense"] = self.data.loc[i, "Cumulative_Cost"] * (self.r * 1/52)
                
                
        #Deal with Profit
        self.Call_Price = self.data["Call_Black_Schole"][0]
        self.Total_Income = (self.Call_Price * self.W * (1 + (self.r * self.T / 52)))
        
        if (self.data.Weekly_Price[len(self.data) - 1]) >= self.K:
            self.Total_Cost = self.data.Cumulative_Cost[len(self.data) - 1] - (self.K * self.W)
        else:
            self.Total_Cost = self.data.Cumulative_Cost[len(self.data) - 1]
        self.Profit = self.Total_Income - self.Total_Cost

#### Calculate profit

In [3]:
%%time

#Initial parameters
W = 1000000
r = 0.01035
T = 20
data = "Weekly_Price.csv"

#Create table to calculate different profit according to different K, Vol given initial parameters
K_Vol = pd.DataFrame()
for K in np.arange(30, 50.5, 0.5):
    for Vol in np.arange(0.05, 1.05, 0.05):
        K_Vol_temp = pd.DataFrame([K, Vol])
        K_Vol = pd.concat([K_Vol, K_Vol_temp], axis = 1)
        
K_Vol = K_Vol.T
K_Vol.columns = ["K", "Vol"]

#Using apply method, send K, Vol into class Worksheet then return profit according to different K, Vol
K_Vol["Profit"] = K_Vol.apply(lambda x : Worksheet("Weekly_Price.csv", W, x["K"], x["Vol"], r, T).Profit, axis = 1)

Wall time: 32.7 s


It only took 32 seconds to calaculate 820 outcomes in OOP way, but it took almost 22 minutes in a normal way(without OOP). However, I am not pretty sure whether the shortness of time taken is due to OOP or not.

**To be continued...**

1. How to calculate profit according to different K, Vol by using the function built inside the class, instead of calculating it by using apply method above. ex. input range of K, Vol, and return profit of it.

2. How to build plotting function inside the class, instead of plotting outside of the class.

3. How to use multiprocessing to speed up calculation process, I had tried a lot of code underneath, but still failed to success. If succeed, still need to think how to write this method into the class.

*Failed code of Multiprocessing:*

In [4]:
# import time
# from concurrent.futures import ProcessPoolExecutor
# from multiprocessing import cpu_count

# def calculate_profit(input_values):
#     K, Vol = input_values
#     return Worksheet("Weekly_Price.csv", W, K, Vol, r, T).Profit

# start = time.time()

# if __name__ == "__main__":
#     pool = ProcessPoolExecutor(max_workers=cpu_count())
#     results = list(pool.map(calculate_profit, K_Vol.values))
#     end = time.time()
#     print('Took %.3f secondes'%(end-start))

In [5]:
# %%time
# from multiprocessing import cpu_count, Pool
# cores = cpu_count() 
# partitions = cores - 1

# def parallelize(data, func):
#     data_split = np.array_split(data, partitions)
#     pool = Pool(cores)
#     data = pd.concat(pool.map(func, data_split))
#     pool.close()
#     pool.join()
#     return data

# def calculate_profit(input_values):
#     K, Vol = input_values
#     return Worksheet("Weekly_Price.csv", W, K, Vol, r, T).Profit

# if __name__ == "__main__":
#     Final_Profit = parallelize(K_Vol.values, calculate_profit)